https://stepik.org/lesson/1576210/step/5

In [ ]:
import torch
import torch.nn as nn

class KLLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, h_mean, h_log_var):
        # h_mean, h_log_var: тензоры размером (batch_size, latent_dim)
        # Вычисляем KL для каждого элемента батча и усредняем
        kl_divergence = -0.5 * torch.sum(1 + h_log_var - h_mean**2 - torch.exp(h_log_var), dim=1).mean()

        return kl_divergence

batch_size = 5
h_mean = torch.rand(batch_size, 10)
h_log_var = torch.rand(batch_size, 10)

model = KLLoss()
model.eval()

loss = model(h_mean, h_log_var).item()
loss

2.5327744483947754

In [ ]:
import torch

def kl_divergence(h_mean, h_log_var):
    # h_mean, h_log_var: тензоры размером (batch_size, latent_dim)
    # Вычисляем KL для каждого элемента батча
    kl = -0.5 * torch.sum(1 + h_log_var - h_mean.pow(2) - torch.exp(h_log_var), dim=1)
    return kl  # размер (batch_size,)

# Пример использования
batch_size = 5
latent_dim = 10
h_mean = torch.rand(batch_size, latent_dim)
h_log_var = torch.rand(batch_size, latent_dim)

kl_values = kl_divergence(h_mean, h_log_var)
print(kl_values)  # KL для каждого элемента батча
print(kl_values.mean())  # Средний KL по батчу


tensor([2.7039, 2.8855, 3.1516, 1.9499, 1.6161])
tensor(2.4614)


Разберём подробно связь между двумя формулами KL-дивергенции и объясним по каждому слагаемому, почему они выглядят по-разному.

---

## Формулы

1) Классическая формула KL-дивергенции между двумя многомерными нормальными распределениями (в частности между $q = \mathcal{N}(\mu_G, \Sigma_G)$ и стандартным нормальным $p = \mathcal{N}(0, I)$):

$$
D_{KL} = \frac{1}{2} \left( \operatorname{tr}(\Sigma_G) + \mu_G^T \mu_G - n - \log \det \Sigma_G \right)
$$

2) Часто используемая в VAE формула, когда ковариационная матрица диагональна и задаётся через вектор логарифмов дисперсий $h_{log\_var}$, а вектор средних $h_{mean}$:

$$
D_{KL}(q(h) \| p(h)) = -\frac{1}{2} \sum_{j=1}^n \left(1 + h_{log\_var,j} - h_{mean,j}^2 - \exp(h_{log\_var,j}) \right)
$$

---

## Связь и разбор по слагаемым


| № | Классическая формула (матрицы) | VAE-формула (поэлементно) | Объяснение |
|-|-|-|-|
|1| $\operatorname{tr}(\Sigma_G)$ | $\sum_{j=1}^n \exp(h_{log\_var,j})$ | След ковариационной матрицы равен сумме диагональных элементов. В VAE ковариационная матрица диагональна, и диагональные элементы - это дисперсии $\sigma_j^2 = \exp(h_{log\_var,j})$. Поэтому $\operatorname{tr}(\Sigma_G) = \sum_j \sigma_j^2 = \sum_j \exp(h_{log\_var,j})$. |
|2| $\mu_G^T \mu_G = \sum_{j=1}^n \mu_{G,j}^2$ | $\sum_{j=1}^n h_{mean,j}^2$ | Вектор средних в классической формуле - это просто сумма квадратов средних по всем признакам. В VAE это то же самое. |
|3| $- n$ | $\sum_{j=1}^n 1$ (внутри скобок: единица) | В классической формуле вычитается размерность $n$, а в VAE формуле в сумме по признакам внутри скобок стоит единица для каждого признака. Это одно и то же: $\sum_{j=1}^n 1 = n$. |
|4| $- \log \det \Sigma_G$ | $\sum_{j=1}^n h_{log\_var,j}$ | Логарифм определителя диагональной матрицы равен сумме логарифмов диагональных элементов: $\log \det \Sigma_G = \sum_j \log \sigma_j^2 = \sum_j h_{log\_var,j}$. В VAE формуле стоит с плюсом, но внутри скобок стоит $- h_{log\_var,j}$, что эквивалентно $- \log \det \Sigma_G$. |

---

## Почему появилась единица и куда делась $n$?

- В классической формуле $-n$ - это вычитание размерности.
- В VAE формуле это реализовано как сумма единиц по размерности: $\sum_{j=1}^n 1 = n$.
- Поэтому единица в скобках - это просто поэлементное представление вычитания размерности.

---

## Откуда взялся квадрат в третьем слагаемом?

- Квадрат - это $\mu_j^2$, квадрат среднего по j-му признаку.
- В классической формуле это $\mu_G^T \mu_G = \sum_j \mu_j^2$.
- В VAE формуле это $- h_{mean,j}^2$ внутри скобок, что соответствует тому же слагаемому.

---

## Откуда взялось $\exp(h_{log\_var,j})$?

- В VAE ковариационная матрица диагональна, и дисперсии задаются через логарифм дисперсии: $h_{log\_var,j} = \log \sigma_j^2$.
- Чтобы получить дисперсию $\sigma_j^2$, нужно взять экспоненту: $\sigma_j^2 = \exp(h_{log\_var,j})$.
- В классической формуле стоит $\operatorname{tr}(\Sigma_G) = \sum_j \sigma_j^2$, в VAE - $\sum_j \exp(h_{log\_var,j})$.

---

## Итог: обе формулы эквивалентны

- Классическая формула - матричная, с ковариационной матрицей $\Sigma_G$.
- VAE-формула - поэлементная, с диагональной ковариационной матрицей, заданной через вектор логарифмов дисперсий.
- Минус перед формулой VAE связан с тем, что часто KL-дивергенция входит в функцию потерь с минусом (см. ELBO), но по сути это тот же KL.

---

Если кратко:

| Классическая формула | VAE-формула | Комментарий |
|-|-|-|
| $\operatorname{tr}(\Sigma_G)$ | $\sum_j \exp(h_{log\_var,j})$ | След ковариации |
| $\mu_G^T \mu_G$ | $\sum_j h_{mean,j}^2$ | Квадрат нормы среднего |
| $-n$ | $\sum_j 1$ | Размерность |
| $-\log \det \Sigma_G$ | $\sum_j -h_{log\_var,j}$ | Логарифм определителя диагональной матрицы |

---

Citations:

[1] https://education.yandex.ru/handbook/ml/article/variational-autoencoder-(vae)

[2] https://education.yandex.ru/handbook/ml/article/entropiya-i-semejstvo-eksponencialnyh-raspredelenij

[3] https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D1%81%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D0%B5_%D0%9A%D1%83%D0%BB%D1%8C%D0%B1%D0%B0%D0%BA%D0%B0_%E2%80%94_%D0%9B%D0%B5%D0%B9%D0%B1%D0%BB%D0%B5%D1%80%D0%B0

[4] https://yu-xuan.livejournal.com/145896.html

[5] https://asu.tusur.ru/learning/090401e/d02/090401e-d02-labs.pdf

[6] https://habr.com/ru/companies/ods/articles/322514/

[7] http://www.machinelearning.ru/wiki/images/6/6f/Yudin20msc.pdf

[8] https://inis.iaea.org/records/5kc0x-fag74/files/16081948.pdf?download=1

---
Answer from Perplexity: pplx.ai/share